# Ryzen AI CVML Integration with ROS

In this notebook, we'll integrate the Ryzen AI CVML library with ROS 2 to create NPU-accelerated vision nodes for robotics applications.

## Goals

* Learn how to integrate NPU acceleration into your own ROS projects
* Launch CVML nodes with ROS 2
* Visualize depth estimation and face detection outputs in ROS

## References

* [Writing a Simple Publisher and Subscriber (C++)](https://docs.ros.org/en/kilted/Tutorials/Beginner-Client-Libraries/Writing-A-Simple-Cpp-Publisher-And-Subscriber.html)
* [Using ROS 2 Launch Files](https://docs.ros.org/en/kilted/Tutorials/Intermediate/Launch/Launch-Main.html)
* [cv_bridge](https://github.com/ros-perception/vision_opencv/tree/ros2/cv_bridge)

## The `ryzen_ai_cvml` ROS Package

The `ryzen_ai_cvml` package wraps the CVML C++ API into ROS 2 nodes. It provides:

### Nodes
- **depth_estimation_node**: Subscribes to image topics, publishes depth maps
- **face_detection_node**: Detects faces and publishes bounding boxes + landmarks
- **face_mesh_node**: Generates 3D face meshes (468 landmarks)

## Explore the Package Structure

Let's look at what's in the `ryzen_ai_cvml` package:

In [1]:
!ls -R cvml_ros/

cvml_ros/:
CMakeLists.txt	include  launch  package.xml  README.md  scripts  src

cvml_ros/include:
cvml_ros

cvml_ros/include/cvml_ros:
depth_estimation_node.hpp  face_detection_node.hpp  face_mesh_node.hpp

cvml_ros/launch:
depth_estimation.launch.py  face_detection.launch.py  face_mesh.launch.py

cvml_ros/scripts:
video_publisher.py

cvml_ros/src:
depth_estimation_node.cpp  face_detection_node.cpp  face_mesh_node.cpp
face_detection_main.cpp    face_mesh_main.cpp	    main.cpp


## Building the ROS Package

If the package isn't built yet, we need to build it with colcon:

In [2]:
%%bash
# This cell might take a few minutes

cd cvml_ros

# Check if already built
if [ -d "build" ]; then
    echo "Package appears to be already built."
    echo "Skipping build. If you need to rebuild, run: colcon build --packages-select cvml_ros"
else
    echo "Building cvml_ros package..."
    source /opt/ros/kilted/setup.bash
    cd ..
    colcon build --packages-select cvml_ros
    echo "Build complete!"
fi

Building cvml_ros package...
Starting >>> cvml_ros
Finished <<< cvml_ros [0.09s]

Summary: 1 package finished [0.31s]
Build complete!


## Launch Loopback Video Publisher

For this part you will want to open a new jupyter terminal

![](images/new_terminal.png)

Copy-paste in the following:

```bash
sudo bash -c "source install/setup.sh && \
              ros2 run cvml_ros video_publisher.py \
                --ros-args \
                -p video_path:=/ryzers/RyzenAI-SW/Ryzen-AI-CVML-Library/samples/video_call.mp4 \
                -p topic:=/camera/image_raw"
```

You should see these messages if successful:
```
[INFO] [1760488629.706959565] [video_publisher]: Publishing video from: /ryzers/RyzenAI-SW/Ryzen-AI-CVML-Library/samples/video_call.mp4
[INFO] [1760488629.707166260] [video_publisher]: Publishing to topic: /camera/image_raw
```

Let's see if ROS detects our new node

In [3]:
!source install/setup.sh && ros2 topic list

/camera/image_raw
/parameter_events
/rosout


Excellent! We have a video_publisher node active in our system. Now let's use the CVML depth estimation node to subscribe to it.

## Launch Depth Estimation Pipeline

For this we will want yet another new terminal

![](images/new_terminal.png)

Now type these commands into the new terminal:

```bash
source /opt/ros/kilted/setup.bash
source install/setup.bash

sudo bash -c "source /opt/ros/kilted/setup.bash && \
              source install/setup.sh && \
              export LD_LIBRARY_PATH=$LD_LIBRARY_PATH && \
              ros2 launch cvml_ros depth_estimation.launch.py"
```

The node will subscribe to our video_publisher and publish depth maps on `/depth/image`. Once all goes well you should observe lots of output, however key lines to look out for are:

```
...
[depth_estimation_node-1] [INFO] [1760488970.246808904] [depth_estimation_node]: Ryzen AI Depth Estimation initialized
[depth_estimation_node-1] [INFO] [1760488970.491575724] [depth_estimation_node]: Created publisher on: /depth_estimation/depth
[depth_estimation_node-1] [INFO] [1760488970.491899477] [depth_estimation_node]: Created subscriber on: /camera/image_raw
[depth_estimation_node-1] [INFO] [1760488970.491906947] [depth_estimation_node]: Depth estimation node started successfully
[depth_estimation_node-1] [INFO] [1760488970.525778512] [depth_estimation_node]: Received first image: 1920x1080
...
```

### Check Active Topics and Nodes

In [20]:
# for some reason when nodes run in sudo we can't see node list
#!source install/setup.sh && ros2 node list

Now for the topics

In [4]:
!source install/setup.sh && ros2 topic list

/camera/image_raw
/depth_estimation/depth
/parameter_events
/rosout


Also, let's make sure the NPU is actually being utilized

In [5]:
!sudo /opt/xilinx/xrt/bin/xrt-smi examine --report aie-partitions


--------------------------------
[0000:c4:00.1] : NPU Strix Halo
--------------------------------
AIE Partitions
  Total Memory Usage: N/A
  Partition Index   : 0
    Columns: [0, 1, 2, 3, 4, 5, 6, 7]
    HW Contexts:
      |PID                 |Ctx ID     |Submissions |Migrations  |Err  |Priority |
      |Process Name        |Status     |Completions |Suspensions |     |GOPS     |
      |Memory Usage        |Instr BO   |            |            |     |FPS      |
      |                    |           |            |            |     |Latency  |
      |====================|===========|============|============|=====|=========|
      |652412              |1          |5653        |0           |0    |Normal   |
      |N/A                 |Active     |5653        |28          |     |9        |
      |N/A                 |1712 KB    |            |            |     |N/A      |
      |                    |           |            |            |     |N/A      |
      |--------------------|------

Great, now we have a mp4 -> video_publisher -> depth_estimation pipeline! What are we missing? We need to visualize the output

## Visualize the Pipeline

```bash
sudo bash -c "source install/setup.sh && \
              ros2 run web_video_server web_video_server --ros-args -p port:=8080 -p address:=0.0.0.0"
```

In [7]:
!source install/setup.sh && ros2 topic list

/camera/image_raw
/depth_estimation/depth
/parameter_events
/rosout


In [8]:
from IPython.display import HTML, display
from ipywidgets import Button, Output
import requests
import base64

out = Output()
display(out)

def update_display(b=None):
    with out:
        out.clear_output(wait=True)
        try:
            camera = requests.get('http://localhost:8080/snapshot?topic=/camera/image_raw', timeout=3).content
            depth = requests.get('http://localhost:8080/snapshot?topic=/depth_estimation/depth', timeout=3).content
            
            cam_b64 = base64.b64encode(camera).decode()
            depth_b64 = base64.b64encode(depth).decode()
            
            display(HTML(f'''
            <div style="display: flex; gap: 20px;">
                <div>
                    <h3>Camera</h3>
                    <img src="data:image/jpeg;base64,{cam_b64}" width="480">
                </div>
                <div>
                    <h3>Depth (NPU)</h3>
                    <img src="data:image/jpeg;base64,{depth_b64}" width="480">
                </div>
            </div>
            '''))
        except Exception as e:
            print(f"Error: {e}")

button = Button(description="Refresh Images")
button.on_click(update_display)
display(button)
update_display()  # Initial display

Output()

Button(description='Refresh Images', style=ButtonStyle())

## Key Takeaways

### Integration Pattern

1. **CVML Context** → Create and configure backend
2. **CVML Feature** → Initialize the vision feature (depth, face, etc.)
3. **ROS Subscriber** → Receive image messages
4. **cv_bridge** → Convert between ROS and OpenCV formats
5. **CVML Processing** → NPU-accelerated inference
6. **ROS Publisher** → Publish results

### Benefits of NPU in ROS

- **Power Efficient**: Lower power draw compared to GPU/CPU
- **Dedicated Hardware**: Frees up CPU/GPU for other tasks
- **Real-time Performance**: ~30 FPS for depth estimation
- **Easy Integration**: Same ROS patterns as any other vision node

### Production Considerations

1. **Backend Selection**: Use parameters to allow runtime backend switching
2. **Error Handling**: Check NPU availability, handle fallback to CPU
3. **Performance Monitoring**: Use `xrt-smi` to verify NPU utilization
4. **Resource Management**: Properly release CVML contexts on shutdown

## Next Steps

You've now learned how to:
- ✅ Validate and use the NPU with `xrt-smi`
- ✅ Run CVML samples on different backends
- ✅ Integrate CVML with ROS 2 for robotics applications

### Further Exploration

- Try different CVML features (face mesh, pose estimation)
- Create custom ROS nodes with CVML integration
- Combine multiple CVML features in a single pipeline
- Benchmark NPU vs GPU vs CPU for your specific workloads
- Build custom NPU applications with MLIR-AIE (advanced)

---
Copyright© 2025 AMD, Inc SPDX-License-Identifier: MIT